In [ ]:
#LFW (Labeled Faces in the Wild) verisetinden yüz tanıma üzerine bir cnn
#modeli eğiteceğiz. Bu model için klasör içerisindeki görsellerin oludğu klasör
#zipini import edelim

from google.colab import files

# ZIP dosyasını yükle
uploaded = files.upload()


Saving lfw-deepfunneled.zip to lfw-deepfunneled.zip


In [ ]:
import zipfile

# Zip dosyasını "lfw_faces" klasörüne açıyoruz
with zipfile.ZipFile("lfw-deepfunneled.zip", 'r') as zip_ref:
    zip_ref.extractall("lfw_faces")

print("ZIP dosyası başarıyla çıkarıldı.")


ZIP dosyası başarıyla çıkarıldı.


In [ ]:
#Klasörde kaç kişi ve kaç görsel olduğunu kontrol etme

import os

klasor_yolu="lfw_faces/lfw-deepfunneled"
kisi_listesi=os.listdir(klasor_yolu)

print(f"Toplam kişi sayısı: {len(kisi_listesi)}")

for kisi in kisi_listesi[:5]:
  foto_sayisi=len(os.listdir(os.path.join(klasor_yolu,kisi)))

  print(f"{kisi}: {foto_sayisi} fotoğraf")

#bir kişinin kaç fotoğrafı olduğunu bulma

Toğlam kişi sayısı: 5749
Latrell_Sprewell: 2 fotoğraf
George_Karl: 2 fotoğraf
Hugo_Chavez: 71 fotoğraf
Francois_Pienaar: 1 fotoğraf
Huang_Suey-Sheng: 1 fotoğraf


In [ ]:
#bılarının az sayıda görseli mevcut bunları ayıracğız modelin iyi eğitilebilmesi için

#En az 15 görseli olan kişileri seçip ayırma

import shutil
kaynak_dizin= "lfw_faces/lfw-deepfunneled"

hedef_dizin="filtered_faces"

os.makedirs(hedef_dizin,exist_ok=True)
#yeni klasör oluşturma

for kisi in os.listdir(kaynak_dizin):
  #kişileri dolaşarak yeterli fortoğrafı olan kişileri hedef dizine atayacağız

  kisi_klasoru=os.path.join(kaynak_dizin,kisi)
  resimler= os.listdir(kisi_klasoru)


  if( len(resimler)) >=15: # en az 15 görseli varsa

       hedef_kisi_klasoru= os.path.join(hedef_dizin,kisi)

       os.makedirs(hedef_kisi_klasoru, exist_ok=True)


       for resim in resimler:
        #görselleri yeni klasöre kopyalama

        kaynak_resim = os.path.join(kisi_klasoru, resim)
        shutil.copy(kaynak_resim,hedef_kisi_klasoru)



print("Yeterli verisi olan kişiler kopyalandı")


Yeterli verisi olan kişiler kopyalandı


In [ ]:
#filtrelenmiş Verilerin Kontrolü

import os

filtreli_dizin="filtered_faces"

kisi_listesi=os.listdir(filtreli_dizin)

print(f"Yeni kişi sayısı (15+ görseli olan kişiler):{len(kisi_listesi)}")


for kisi in kisi_listesi[:5]:
  foto_sayisi= len(os.listdir(os.path.join(filtreli_dizin,kisi)))

  print(f"{kisi}: {foto_sayisi} görsel")

#ilk 5 kişinin kaç görsel olduğunu gösterme

Yeni kişi sayısı (15+ görseli olan kişiler):96
Hugo_Chavez: 71 görsel
Igor_Ivanov: 20 görsel
Ariel_Sharon: 77 görsel
Renee_Zellweger: 17 görsel
Jack_Straw: 28 görsel


In [ ]:
#Eğitim ve Test Verileri Ayrımı


import os
import random
import shutil
from sklearn.model_selection import train_test_split

kaynak_dizin = "filtered_faces"
train_dizin= "data_split/train"
val_dizin = "data_split/val"

os.makedirs(train_dizin , exist_ok=True)
os.makedirs(val_dizin, exist_ok=True)

#her kişi için işlemi yapacağız görsellerinin bir kısmı  eğitim
#bir kısmı doğrulama verisi olacak

for kisi in os.listdir(kaynak_dizin):
  resimler = os.listdir(os.path.join(kaynak_dizin, kisi))

  train_imgs, val_imgs = train_test_split(resimler, test_size=0.2, random_state=42)
  #eğitim ve doğrulama olarak ayrılması

  os.makedirs(os.path.join(train_dizin,kisi),exist_ok=True)
  os.makedirs(os.path.join(val_dizin, kisi),exist_ok=True)
  #kişilerin ayrılan görselleri için klasörler oluşturma

   #eğitim verilerini kalsörlerine kopyalama
  for img in train_imgs:
    shutil.copy(os.path.join(kaynak_dizin,kisi,img),
                os.path.join(train_dizin,kisi,img))

  #Doğrulama verilerini klasörlerine kopyalama

  for img in val_imgs:
    shutil.copy(os.path.join(kaynak_dizin,kisi,img),
                os.path.join(val_dizin, kisi, img))


print("Veri başarıyla eğitim ve doğrulama için ayrıldı")

Veri başarıyla eğitim ve doğrulama için ayrıldı


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
#ImageDataGenerator ile Veri Okuma

train_datagen=ImageDataGenerator(rescale =1./255)#0-255 arası görselleri 1-0 arası
#normalize eder
val_datagen=ImageDataGenerator(rescale=1./255)

#Eğitim verisini okuma

train_generator = train_datagen.flow_from_directory(
    "data_split/train", #okuyacağı klasör dizini
    target_size=(100,100),#resimlerin geleceği boyut
    batch_size=32,
    class_mode='categorical' #sınıflandırma türü
)

#doğrulama verisini okuma

val_generator= val_datagen.flow_from_directory(
    "data_split/val",#okuyacağı klasör dizini
    target_size=(100,100), #resimleri boyutu
    batch_size=32,
    class_mode='categorical' # sınıflandırma türü
)

Found 2839 images belonging to 96 classes.
Found 756 images belonging to 96 classes.


In [ ]:
#CNN Modeli Kurulumu ve Eğitimi

model=tf.keras.Sequential()
#sıfırdan cnn modeli inşası


model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))

#ilk konvolüsyon katmani özellik çıkarımı
model.add(MaxPooling2D(pool_size=(2,2)))
#boyutu küçültür,bilgi yoğunluğunu arttırr

model.add(Conv2D(64,(3, 3),activation='relu'))
#2. katman
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#3. katman

model.add(Flatten())
#modeli düzleştirme 2D den 1D ye

model.add(Dense(128,activation='relu')) #tam bağlantılı katman
model.add(Dropout(0.5))#overfittingi önlemek için


model.add(Dense(train_generator.num_classes, activation='softmax'))
#çıkış katmanı

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#modeli derleme


model.summary()#modelin özetini çıkarma

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 98, 98, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 49, 49, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 47, 47, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 21, 21, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,638,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 96)             │        12,384 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,744,160 (6.65 MB)

 Trainable params: 1,744,160 (6.65 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#Modeli Eğitme

from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss',patience=3, restore_best_weights=True)

history = model.fit(
    train_generator,
    epochs=15 ,
    validation_data=val_generator,
    callbacks=[early_stop]
)

Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


89/89 ━━━━━━━━━━━━━━━━━━━━ 68s 728ms/step - accuracy: 0.1369 - loss: 4.3157 - val_accuracy: 0.1402 - val_loss: 3.9982
Epoch 2/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 68s 763ms/step - accuracy: 0.1546 - loss: 3.9956 - val_accuracy: 0.1720 - val_loss: 3.8372
Epoch 3/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 77s 706ms/step - accuracy: 0.1988 - loss: 3.6923 - val_accuracy: 0.2209 - val_loss: 3.4783
Epoch 4/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 63s 706ms/step - accuracy: 0.2273 - loss: 3.4206 - val_accuracy: 0.2791 - val_loss: 3.1565
Epoch 5/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 83s 715ms/step - accuracy: 0.3220 - loss: 2.9227 - val_accuracy: 0.3611 - val_loss: 2.7636
Epoch 6/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 61s 683ms/step - accuracy: 0.3602 - loss: 2.6458 - val_accuracy: 0.4034 - val_loss: 2.5943
Epoch 7/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 63s 708ms/step - accuracy: 0.4140 - loss: 2.3396 - val_accuracy: 0.4140 - val_loss: 2.4441
Epoch 8/15
89/89 ━━━━━━━━━━━━━━━━━━━━ 63s 714ms/step - accuracy: 0.4835 - loss: 2.0158 - val_accuracy: 0.493

In [ ]:
model.save("lfw_yuz_tanima_modeli.h5")

In [ ]:
#Modeli indirme

from google.colab import files

files.download("lfw_yuz_tanima_modeli.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>